## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [7]:
df=pd.read_csv('train.csv')

In [8]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [9]:
# Check for null values
df.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [10]:
df.shape

(23196, 5)

In [11]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [12]:
df.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [13]:
###Drop Nan Values since null records are less
df=df.dropna()


In [14]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [18]:
## Get the Independent Features by dropping label
X=df.drop('real',axis=1)

In [19]:
## Get the Dependent features
y=df['real']

In [20]:
X.shape

(22866, 4)

In [21]:
y.shape

(22866,)

In [22]:
import tensorflow as tf

In [23]:
tf.__version__

'2.19.0'

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [25]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [30]:
# Make a copy
messages=X.copy()

In [31]:
messages['title'][1]

"People's Choice Awards 2018: The best red carpet looks"

In [32]:
messages

,title,news_url,source_domain,tweet_num
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [33]:
messages.reset_index(inplace=True)

In [34]:
messages

,index,title,news_url,source_domain,tweet_num
0,0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...,...
22861,23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
22862,23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
22863,23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
22864,23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [35]:
import nltk
import re
from nltk.corpus import stopwords

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [39]:
### Dataset Preprocessing - StopWords & Stemming
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [40]:
corpus

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang daughter kati complet leav son b

In [41]:
corpus[1]

'peopl choic award best red carpet look'

In [43]:
# Extract one hot encoded representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] # Get indexes of 1
onehot_repr

[[4084, 1020, 126, 2882, 208, 4545, 2300, 2730, 3074, 1860],
 [4392, 1766, 4605, 1452, 2651, 3524, 3554],
 [979,
  627,
  796,
  2891,
  4259,
  1486,
  2282,
  3742,
  263,
  618,
  443,
  1224,
  1199,
  720,
  2272],
 [680, 2834, 1104, 1222, 4174, 2638, 3293, 3163],
 [440, 4205, 4827, 2350, 1030, 3660, 764, 332, 1220, 703, 3827, 3392, 3843],
 [502, 3744, 4992, 912, 3490, 3539, 1044, 2237, 2896],
 [745, 4758, 3437, 2017, 599, 3951],
 [356, 1177, 4732, 536, 1658, 3130, 2606, 2601, 4458, 4488],
 [1309, 393, 2523, 4839, 1056, 1461, 1136, 3209, 878, 890, 618, 443],
 [3956, 1377, 1140, 4788, 2080, 3973],
 [3213, 3071, 443, 3912, 3654, 1555, 2430, 1645, 3223],
 [1982, 4234, 3805, 2788, 1789, 2060, 3002],
 [4658, 4889, 1876, 58, 3647, 1486, 1690, 3805, 4000, 245, 3785, 537],
 [1679, 3644, 1337, 124, 346, 3765],
 [906, 1617, 3534, 3594, 362, 4369, 4297, 3381],
 [824, 2172, 2909, 3182, 1691, 2077, 2878, 2664, 4144, 1170],
 [385, 2865, 1873, 3930, 2129, 2369, 2516],
 [2504, 2247, 4172, 1013, 5

In [44]:
corpus[1]

'peopl choic award best red carpet look'

In [45]:
onehot_repr[1]

[4392, 1766, 4605, 1452, 2651, 3524, 3554]

### Embedding Representation

In [46]:
# Apply Post Padding
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4084 1020  126 ...    0    0    0]
 [4392 1766 4605 ...    0    0    0]
 [ 979  627  796 ...    0    0    0]
 ...
 [ 611 1007 1293 ...    0    0    0]
 [ 752 4381  508 ...    0    0    0]
 [3951  749 4707 ...    0    0    0]]


In [47]:
embedded_docs[1]

array([4392, 1766, 4605, 1452, 2651, 3524, 3554,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [48]:
embedded_docs[0]

array([4084, 1020,  126, 2882,  208, 4545, 2300, 2730, 3074, 1860,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [49]:
## Creating model
embedding_vector_features=40 ##features representation - 40 dimensions
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))  # LSTM layer
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [50]:
len(embedded_docs),y.shape

(22866, (22866,))

In [51]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [52]:
X_final.shape,y_final.shape

((22866, 20), (22866,))

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [54]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7763 - loss: 0.5357 - val_accuracy: 0.8330 - val_loss: 0.4020
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8518 - loss: 0.3565 - val_accuracy: 0.8134 - val_loss: 0.4084
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8717 - loss: 0.3134 - val_accuracy: 0.8237 - val_loss: 0.4006
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8876 - loss: 0.2846 - val_accuracy: 0.8032 - val_loss: 0.4429
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8973 - loss: 0.2646 - val_accuracy: 0.8157 - val_loss: 0.4621
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9040 - loss: 0.2401 - val_accuracy: 0.8177 - val_loss: 0.4728
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9092 - loss: 0.2122 - val_accuracy: 0.8105 - val_loss: 0.5250
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9101 - loss: 0.1943 - val_accuracy: 0.

### Adding Dropout

In [55]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [56]:
y_pred=model.predict(X_test)

236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [57]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [58]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_test,y_pred)

array([[1842,    0],
       [5704,    0]])

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.24410283593957063

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.24      1.00      0.39      1842
           1       0.00      0.00      0.00      5704

    accuracy                           0.24      7546
   macro avg       0.12      0.50      0.20      7546
weighted avg       0.06      0.24      0.10      7546



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
